In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
datos=pd.read_csv('D:/Python/Kaggle/video-games-sales/vgsales-12-4-2019.csv')


In [ ]:
datos.head()

# Preparación de los datos

In [ ]:
#Nueva variable
datos['producido']=datos['Total_Shipped']
datos.loc[datos['Total_Shipped'].isna(),'producido']=datos.loc[datos['Total_Shipped'].isna(),'Global_Sales']


In [2]:
indJ=datos['JP_Sales'].isna()
indN=datos['NA_Sales'].isna()
indP=datos['PAL_Sales'].isna()
indO=datos['Other_Sales'].isna()

NameError: name 'datos' is not defined

In [ ]:
#Añadir zona faltante JP
datos.loc[indJ,'JP_Sales']=datos.loc[indJ,'Global_Sales']-datos.loc[indJ,'NA_Sales']-datos.loc[indJ,'PAL_Sales']-datos.loc[indJ,'Other_Sales']
datos.loc[datos['JP_Sales']<0.01,'JP_Sales']=0
#Añadir zona faltante NA
datos.loc[indN,'NA_Sales']=datos.loc[indN,'Global_Sales']-datos.loc[indN,'JP_Sales']-datos.loc[indN,'PAL_Sales']-datos.loc[indN,'Other_Sales']
datos.loc[datos['NA_Sales']<0.01,'NA_Sales']=0
#Añadir zona faltante PAL
datos.loc[indP,'PAL_Sales']=datos.loc[indP,'Global_Sales']-datos.loc[indP,'NA_Sales']-datos.loc[indP,'JP_Sales']-datos.loc[indP,'Other_Sales']
datos.loc[datos['PAL_Sales']<0.01,'PAL_Sales']=0
#Añadir zona faltante Other
datos.loc[indO,'Other_Sales']=datos.loc[indO,'Global_Sales']-datos.loc[indO,'NA_Sales']-datos.loc[indO,'PAL_Sales']-datos.loc[indO,'JP_Sales']
datos.loc[datos['Other_Sales']<0.01,'Other_Sales']=0

In [ ]:
#Añadir indicador multiplataforma
indD=datos['Name'].duplicated(keep=False)
datos['Multiplataforma']=0
datos.loc[indD,'Multiplataforma']=1

# Analisis por plataforma y generacion

### Plataformas con mayor número de juegos vendidos

Una de las problematicas que presenta este dataset es que los valores de venta tienen como orden millones. Eso implica que cuando tenemos valores mas pequeños que los 0.00 millones, no podemos tener información del valor real, unicamente podemos saber que juegos son más vendidos que los otros. 

El primer punto a tratar es estudiar que plataformas son las que venden un mayor numero de juegos. 

In [ ]:
b=datos.groupby(by=['Platform'])['producido'].sum().sort_values(ascending=False).to_frame()
b_others=b[18:].sum()
b.reset_index(inplace=True)
b['Platform'][18]='Others'
b['producido'][18]=b_others
plt.figure(figsize=(10, 8))
plt.pie(b['producido'][:19], labels=b['Platform'][:19], autopct='%1.1f%%', shadow=True, startangle=90)

Viendo el comportamiento historico a priori no hay una gran diferencia entre las cuatro grandes plataformas de videojuegos,(PS, Xbox, PC, Nintendo) a excepción de ps que entre la generacion 2 y 3 suman mas de un 20% de los juegos vendidos. Esta visión tiene en cuenta todos los datos historicos, lo cual no es un analisis adecuado puesto que estamos teniendo en cuenta consolas obsoletas. Nos centraremos ahora en visualizar los ultimos cinco años.

### Plataformas con mayor número de juegos vendidos en los ultimos años

In [ ]:
datos_actuales=datos[(datos['Year']>2014)&(datos['producido'].isna()==False)]

In [ ]:
b=datos_actuales.groupby(by=['Platform'])['producido'].sum().sort_values(ascending=False).to_frame()
b_others=b[7:].sum()
b.reset_index(inplace=True)
b['Platform'][7]='Others'
b['producido'][7]=b_others
plt.figure(figsize=(10, 8))
plt.pie(b['producido'][:8], labels=b['Platform'][:8], autopct='%1.1f%%', shadow=True, startangle=90)

El comportamiento es claramente distinto, como podemos ver los juegos mas vendidos publicados en los ultimos años, son aquellos que utilizan las plataformas de ultima generación, siendo ps4 la clara ganadora de todas. 

### Plataforma vs multiplataforma

Para determinar cual de las dos opciones es la mas util, vamos a comparar el comportamineto de en el ranking en funcion de si el juego es multiplataforma o no. para ello utilizaremos un estimador no parametrico de la funcion de densidad. La idea aqui es ver de forma grafica cual de las dos opciones tiene una mayor probabilidad de exito o fracaso.

 Las funciones de densidad no estan escalada, lo importante no son tanto los valores como las formas. En ambos casos tenemos dos modas, que en cierta medida las podriamos ver como exito y fracaso. A priori los videojuegos de una sola plataforma parecen tener una mayor probabilidad de exito. Para ver esto un poco mas en detalle vamos ahora a estudiar este caso para cada una de las cuatro grandes plataformas.

In [ ]:
datos_actuales.loc[datos_actuales['ESRB_Rating'].isna(),'ESRB_Rating']='SC'

In [ ]:
d = {'producido':'producido', 'Multiplataforma':'Multiplataforma'}
datos_platf=datos_actuales.groupby(['Name','Genre','Publisher','ESRB_Rating']).agg({'producido':'sum','Multiplataforma':'mean'}).rename(columns=d)
datos_platf=datos_platf.reset_index()
datos_platf=datos_platf.sort_values(by=['producido'], ascending=False)
datos_platf['Rank']=np.arange(datos_platf.shape[0])+1

In [ ]:
datos_platf

In [ ]:
plt.figure(figsize=(15, 10))
sns.displot(datos_platf, x="Rank", col="Multiplataforma", kind="kde", fill=True)


In [ ]:
datos_actuales_plat=datos_actuales[(datos_actuales['Platform']=='PS4')| (datos_actuales['Platform']=='PC')| (datos_actuales['Platform']=='XOne')| (datos_actuales['Platform']=='NS')]

In [ ]:
plt.figure(figsize=(15, 10))
sns.displot(datos_actuales_plat, x="Rank", hue="Multiplataforma", kind="kde",col="Platform", fill=True)

A excepcion de la xbox que apenas tiene titulos propios, seguimos teniendo esa estructura de de dos modas. De apostar por una plataforma esa seria ps. Visto estos resultados uno podria pensar que la mejor opcion seria apostar por un juego para ps, sin embargo hay una serie de detalles que conviene mencionar. EL hecho de que no tengamos toda la informacion de las ventas reales hace que los juego multiplataforma no se puedan reagrupar. Eso genera un sesgo negativo en el analisis de los juegos multiplataforma. Lo segundo pero no menos importante es que las plataformas invierten mucho dinero en conseguir exclusividad en ciertos juegos con una fanbase  grande. Esto genera que parte de los juegos exitos exclusivos a una plataforma son exitos no por ser exclusivos si no por tener un gran numero de seguidores. 

La decision de elegir si apostar por una plataforma o multiplataforma depende tambien del incremento en el coste que pueda suponer desarrollar un juego en varias plataformas. 

# Analisis por Genero

Ya hemos determinado que a priori la mejor decision podria ser desarrollar un juego multiplataforma, sin embargo, este resultado no es 100% seguro si no estudiamos previamente los generos de videojuegos mas demandados, y si esos generos son comunes para las distintas plataformas. No tendria ningun sentido desarrollar un juego multiplataforma cuando solo tiene demanda para una plataforma.

### Que genero de videojuegos es el mas demandado

In [ ]:
datos_genre = datos_actuales.groupby(by=['Genre'])['producido'].sum()
datos_genre = datos_genre.reset_index()
datos_genre['count']=datos_platf['Genre'].value_counts().reset_index().sort_values(by=['index'])['Genre'].values
datos_genre = datos_genre.sort_values(by=['producido'], ascending=False)
# data_genre

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Genre", y="producido", palette="Set2",data=datos_genre)
plt.xticks(rotation=90)

Estos son los generos que mas ventas generan. Sin embargo, un genero puede tener un numero muy grande de ventas y que el numero de juegos publicados sea muy grande, por lo que implica una mayor competencia y una menor ganancia, mientras que otro genero que haya generado un numero menor de ventas pero con una cantidad mucho menor de juegos puede generar una mayor ganancia. Lo que haremos ahora es estudiar la ganancia media por genero.

In [ ]:
datos_genre['producido']=datos_genre['producido']/datos_genre['count']
datos_genre = datos_genre.sort_values(by=['producido'], ascending=False)

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Genre", y="producido", palette="Set2",data=datos_genre)
plt.xticks(rotation=90)

### Analisis por cada una de las cuatro consolas principales 

Como hemos visto los shooter son la categoria que destaca sobre todas, pero tiene el mismo  impacto en todas las consolas?. El siguiente paso es estudiar cada una de estas categorias en funcion de la plataforma.

In [ ]:
datos_ps4=datos_actuales[datos_actuales['Platform']=='PS4']
datos_pc=datos_actuales[datos_actuales['Platform']=='PC']
datos_xone=datos_actuales[datos_actuales['Platform']=='XOne']
datos_ns=datos_actuales[datos_actuales['Platform']=='NS']

In [ ]:
#PS4
datos_genre_ps4 = datos_ps4.groupby(by=['Genre'])['producido'].sum()
datos_genre_ps4 = datos_genre_ps4.reset_index()
datos_genre_ps4 = datos_genre_ps4.sort_values(by=['producido'], ascending=False)
#PS4
datos_genre_pc = datos_pc.groupby(by=['Genre'])['producido'].sum()
datos_genre_pc = datos_genre_pc.reset_index()
datos_genre_pc = datos_genre_pc.sort_values(by=['producido'], ascending=False)
#PS4
datos_genre_xone = datos_xone.groupby(by=['Genre'])['producido'].sum()
datos_genre_xone = datos_genre_xone.reset_index()
datos_genre_xone = datos_genre_xone.sort_values(by=['producido'], ascending=False)
#PS4
datos_genre_ns = datos_ns.groupby(by=['Genre'])['producido'].sum()
datos_genre_ns = datos_genre_ns.reset_index()
datos_genre_ns = datos_genre_ns.sort_values(by=['producido'], ascending=False)

Ver plot volcanic

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Genre", y="producido", data=datos_genre_ps4)
plt.xticks(rotation=90)


In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Genre", y="producido", data=datos_genre_pc)
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Genre", y="producido", data=datos_genre_xone)
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Genre", y="producido", data=datos_genre_ns)
plt.xticks(rotation=90)

# Analisis por mercado

El estudio de los mercados esta ligeramente sesgado, puesto que existen un numero de juegos importante en el que no sabes que cantidad que ha demandado cada uno. Existen algun dataset alternativo, sin embargo en muchos casos son contradictorios con el propio dataset.+

In [ ]:
datos_actuales_genre = datos_actuales[['Genre', 'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales']]
# comp_genre
comp_map = datos_actuales_genre.groupby(by=['Genre']).sum()
comp_map=comp_map.sort_values(by=['NA_Sales'], ascending=False)
comp_table = comp_map.reset_index()
comp_table = pd.melt(comp_table, id_vars=['Genre'], value_vars=['NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales'], var_name='Sale_Area', value_name='Sale_Price')
a=comp_table.groupby(by=['Sale_Area']).sum().reset_index()
a=a.sort_values(by=['Sale_Price'], ascending=False)
plt.figure(figsize=(15, 10))
sns.barplot( y='Sale_Price', x='Sale_Area', data=a)
plt.xticks(rotation=70)

### Por genero

In [ ]:
# comp_map
plt.figure(figsize=(15, 10))
sns.set(font_scale=1)
sns.heatmap(comp_map, annot=True, fmt = '.1f')

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x='Genre', y='Sale_Price', hue='Sale_Area', palette="Set2", data=comp_table)
plt.xticks(rotation=70)


El genero de shooter es posiblemente la mejor opcion a seguir, pero con la consecuencia de perder el mercado japones. La segunda mejor opcion es el genero sport y ocurre lo mismo que con el shooter. A pesar de la perdida del mercado japones, estas dos categorias siguen siendo las lideres.  Por otro lado generos como action y role tienen grandes ventas en todos los mercados, aunque cabe destacar que estos dos generos tenian mayor competencia en funcion de a los beneficios, con lo que no son peor opcion que los shooter y sport.

# Analisis top ventas

In [ ]:
datos_shooter=datos_platf[datos_platf['Genre']=='Shooter']


In [ ]:
datos_shooter_ESRB=datos_shooter[datos_shooter['ESRB_Rating'].isna()==False]
datos_shooter_nESRB=datos_shooter[datos_shooter['ESRB_Rating'].isna()]


In [ ]:
sns.displot(datos_shooter_ESRB, x="Rank", col='ESRB_Rating', kind="kde", fill=True)


In [ ]:
datos_shooter_ESRB_pu = datos_shooter_ESRB.groupby(by=['Publisher'])['producido'].sum()
datos_shooter_ESRB_pu= datos_shooter_ESRB_pu.reset_index()
datos_shooter_ESRB_pu = datos_shooter_ESRB_pu.sort_values(by=['producido'], ascending=False)
plt.figure(figsize=(15, 10))
sns.barplot(x="Publisher", y="producido", data=datos_shooter_ESRB_pu.iloc[0:15,:],ci=None)
plt.xticks(rotation=90)

In [ ]:
datos_shooter_final=datos_shooter_ESRB[((datos_shooter_ESRB['Publisher']=='Activision')|(datos_shooter_ESRB['Publisher']=='Electronic Arts'))&(datos_shooter_ESRB['ESRB_Rating']=='M')]

In [ ]:
datos_shooter_final['producido'].mean()

In [ ]:
sns.displot(datos_shooter_final, x="Rank", kind="kde", fill=True)

In [ ]:
sns.displot(datos_shooter_final, x="producido", kind="kde", fill=True)